In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from scipy import io
from tensorflow.keras.callbacks import ModelCheckpoint

In [25]:
dataset = io.loadmat('datasets/emnist-letters.mat')
# matlab_structs / numpy structured arrays. yay
# they have a dtype (structure) and every element adheres to it :)

In [26]:
struct_dataset = dataset['dataset'][0,0]

train_images, train_labels = struct_dataset['train'][0,0]['images'], struct_dataset['train'][0,0]['labels']
test_images, test_labels = struct_dataset['test'][0,0]['images'], struct_dataset['test'][0,0]['labels']

print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)

(124800, 784) (20800, 784) (124800, 1) (20800, 1)


In [27]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding="same", use_bias="false", input_shape=(28,28,1)),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(32, activation="relu"),
    layers.Dense(26, activation="softmax")
])

model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = train_labels - 1
test_labels = test_labels - 1

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

checkpoint = ModelCheckpoint(filepath="checkpoints/epoch_{epoch:02d}.keras", save_weights_only=False, save_freq="epoch")

model.fit(train_images, train_labels, epochs=15,
          validation_data=(test_images, test_labels), callbacks=[checkpoint], batch_size=128)

# Best = 94.62% validation acc

Epoch 1/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 170s 173ms/step - accuracy: 0.7780 - loss: 0.7139 - val_accuracy: 0.9204 - val_loss: 0.2471
Epoch 2/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 169s 173ms/step - accuracy: 0.8942 - loss: 0.3282 - val_accuracy: 0.9206 - val_loss: 0.2526
Epoch 3/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 176s 181ms/step - accuracy: 0.9087 - loss: 0.2793 - val_accuracy: 0.9305 - val_loss: 0.2211
Epoch 4/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 161s 165ms/step - accuracy: 0.9179 - loss: 0.2511 - val_accuracy: 0.9376 - val_loss: 0.1959
Epoch 5/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 160s 164ms/step - accuracy: 0.9241 - loss: 0.2317 - val_accuracy: 0.9357 - val_loss: 0.1985
Epoch 6/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 159s 163ms/step - accuracy: 0.9283 - loss: 0.2189 - val_accuracy: 0.9391 - val_loss: 0.1951
Epoch 7/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 158s 162ms/step - accuracy: 0.9316 - loss: 0.2066 - val_accuracy: 0.9424 - val_loss: 0.1836
Epoch 8/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 159s 163ms/step - accuracy: 0.9340 -